In [1]:
import sys
import platform
from os.path import join, exists, abspath, dirname
from os import getcwd, makedirs
from glob import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colormaps, cm, colors
import matplotlib.patches as patches
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset
import scipy
import seaborn as sns

from tqdm import tqdm

In [2]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GroupKFold, cross_val_score

In [3]:
import nibabel as nb
from nilearn import plotting, image
from nilearn.maskers import NiftiLabelsMasker
from nilearn.decoding import Decoder
from nipype.interfaces import fsl

In [4]:
import subprocess
import warnings

In [5]:
dir_current = getcwd().replace('\\','/')

tmp = dir_current.split('/')
idx = [ii for ii, s in enumerate(tmp) if s=='github'][0]

dir_git = '/'.join(tmp[:idx+1])
dir_git

'/home/sungbeenpark/github'

In [6]:
dname = join(dir_git,'nitools')
sys.path.append(dname)
import nitools as nt

In [7]:
dname = join(dir_git,'SUITPy')
sys.path.append(dname)
import SUITPy as suit

In [8]:
dname = join(dir_git)
sys.path.append(dname)
import surfAnalysisPy as surf

In [9]:
dname = join(dir_git,'SeqSpatialSupp_fMRI')
sys.path.append(dname)
from SSS import deal_spm
from SSS import util as su
from SSS import stat as sstat
from SSS import plot as splt
from SSS import image as simage
from SSS import glmsingle as ssingle

---

In [10]:
dir_root = su.get_dir_root()
dir_root

'/mnt/f/SeqSpatialSupp_fMRI'

In [11]:
list_sn = su.get_list_sn()

---

In [12]:
glm = 1
dir_glm = ssingle.get_dir_glmsingle(glm)

In [13]:
dir_surf = join(ssingle.get_dir_glmsingle(),'surfaceWB')
dir_surf

'/mnt/f/SeqSpatialSupp_fMRI/GLMsingle/surfaceWB'

In [14]:
dir_group = join(dir_surf,'group')

---

In [15]:
dir_fs = su.get_dir_fs()

In [16]:
border, brdr = simage.get_border(join(dir_git,'surfAnalysisPy'))
hem = 'L'

---

In [17]:
## the label list of benson14
list_roi = ['V1','V2','V3','hV4','VO1','VO2','LO1','LO2','TO1','TO2','V3b','V3a']

---

## MVPA

In [18]:
def fast_masking(img_data, img_mask, roi):
    if isinstance(img_data,str):
        img_data = nb.load(img_data)
    img_data = img_data.get_fdata()

    if isinstance(img_mask,str):
        img_mask = nb.load(img_mask)
    img_mask = img_mask.get_fdata().astype(int)
    assert img_data.shape[:3] == img_mask.shape
    
    list_roi_ = np.array(list_roi)
    idx = int(np.where(list_roi_==roi)[0][0])
    assert isinstance(idx, int)
    
    idx_mask = img_mask==idx+1
    data = img_data[idx_mask].T
    
    return data

In [23]:
lines = {'subj':[], 'roi':[], 'score':[]}

for ii, sn in enumerate(tqdm(list_sn)):
    subj = 'S'+sn

    ## labels
    reginfo = deal_spm.load_reginfo(subj, dir_glm)
    y = reginfo.name.values
    y = np.array(['spatial' if 'S' in cue else ('letter' if 'L' in cue else 'invalid') for cue in y])
    group = reginfo.run.values

    ## beta
    list_img_data = sorted(glob(join(dir_glm,subj,'beta_*.nii')))

    ## mask
    dir_roi = join(dir_fs, subj, 'mri')
    img_mask = image.resample_to_img(
        source_img=join(dir_roi,'lh.benson14_varea.vol.nii'),
        target_img=list_img_data[0],
        interpolation='nearest',
        force_resample=True,
        copy_header=True
    )
    
    ## get masked beta
    for roi in list_roi:
        dataset = []
        for img_data in list_img_data:
            data = fast_masking(img_data, img_mask, 'V1')
            dataset.append(data)
        dataset = np.array(dataset)

        ## pipeline: scaling + SVM
        clf = make_pipeline(
            SimpleImputer(strategy='constant', fill_value=0, keep_empty_features=True),
            StandardScaler(),
            SVC(kernel='linear', C=1)
        )

        ## group fold
        gkf = GroupKFold(n_splits=len(np.unique(group)))

        ## run cross-validation
        scores = cross_val_score(estimator=clf, X=dataset, y=y, cv=gkf, groups=group)
        
        ## restore the result
        lines['subj'].append(subj)
        lines['roi'].append(roi)
        lines['score'].append(np.mean(scores))

df = pd.DataFrame(lines)

In [ ]:
df

---

---